In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import random
%matplotlib inline
import tensorflow as tf
import shutil
import tensorflow.contrib.learn as tflearn
import tensorflow.contrib.layers as tflayers
from tensorflow.contrib.learn.python.learn import learn_runner
import tensorflow.contrib.metrics as metrics
import tensorflow.contrib.rnn as rnn

In [ ]:
# x scaling
x_scaler = MinMaxScaler().fit(x_train)
x_train = x_scaler.transform(x_train) # 他是把整个数据集分成 x y  y是目标值 

# y scaling
y_scaler = MinMaxScaler().fit(data)
data = y_scaler.transform(data)

In [ ]:
num_periods = 33 #这个改一下 反正你分割一下 这个 代表有多少个时间周期
f_horizon = 1  #forecast horizon, one period into the future //预测下一个时间周期

x_data = x_train[:(len(x_train)-(len(x_train) % num_periods))] 
x_batches = x_data.reshape(-1, 33, 4) #得到training set的x 后面这个四对应x维数

y_data = data[:(len(x_train)-(len(x_train) % num_periods))]
y_batches = y_data.reshape(-1, 33, 1)#得到training set的y y是目标值


In [ ]:
def test_data(series,forecast,num_periods):
    test_x_setup = x_train[-(num_periods + forecast):]
    testX = test_x_setup[:num_periods].reshape(-1, 33, 4)
    testY = data[-(num_periods):].reshape(-1, 33, 1)
    return testX,testY#得到test set的 x 和y

X_test, Y_test = test_data(data,f_horizon,num_periods )


In [ ]:
tf.reset_default_graph()   #We didn't have any previous graph objects running, but this would reset the graphs

num_periods = 33      #number of periods per vector we are using to predict one period ahead
inputs = 4           #number of vectors submitted 这个看你train 的x 是几维的 是几维就设成几
hidden = 100          #number of neurons we will recursively work through, can be changed to improve accuracy
output = 1            #number of output vectors

X = tf.placeholder(tf.float32, [None, num_periods, inputs])   #create variable objects
y = tf.placeholder(tf.float32, [None, num_periods, output])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=hidden, activation=tf.nn.relu)   #create our RNN object
rnn_output, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)               #choose dynamic over static

learning_rate = 0.001   #small learning rate so we don't overshoot the minimum

stacked_rnn_output = tf.reshape(rnn_output, [-1, hidden])           #change the form into a tensor
stacked_outputs = tf.layers.dense(stacked_rnn_output, output)        #specify the type of layer (dense)
outputs = tf.reshape(stacked_outputs, [-1, num_periods, output])          #shape of results
 
loss = tf.reduce_sum(tf.square(outputs - y))    #define the cost function which evaluates the quality of our model
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)          #gradient descent method
training_op = optimizer.minimize(loss)          #train the result of the application of the cost_function                                 
init = tf.global_variables_initializer()

In [ ]:
epochs = 1000    #number of iterations or training cycles, includes both the FeedFoward and Backpropogation

with tf.Session() as sess:
    init.run()
    ans=[]
    for ep in range(epochs):
        ans=sess.run(training_op, feed_dict={X: x_batches, y: y_batches})
        mse = loss.eval(feed_dict={X: x_batches, y: y_batches})
        if ep % 10 == 0:
            print(ep, "\tMSE:", mse)
    test_pred = sess.run(outputs, feed_dict={X: X_test}) #得到test 结果
    train_pred = sess.run(outputs, feed_dict={X: x_batches})#得到train的结果
    print(y_pred)